In [11]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn  # rnn stuff temporarily in contrib, moving back to code in TF 1.1
import os
import time
import math
import numpy as np
import my_txtutils as txt
import sys

tf.set_random_seed(0)
tf.reset_default_graph()

SEQLEN = 30
BATCHSIZE = 20
ALPHABETSIZE = txt.ALPHASIZE
INTERNALSIZE = 128
NLAYERS = 1


keep_prob = 0.8    # some dropout


# Load Shakespeare Data
shakedir = "shakespeare/*.txt"
codetext, valitext, bookranges = txt.read_data_files(shakedir, validation=True)

# display some stats on the data
epoch_size = len(codetext) // (BATCHSIZE * SEQLEN)
txt.print_data_stats(len(codetext), len(valitext), epoch_size)

# DEFINE PLACEHOLDERS
keep_prob_placeholder = tf.placeholder(tf.float32, name='keep_prob')  # dropout parameter
batchsize = tf.placeholder(tf.int32, name='batchsize')


# inputs
X = tf.placeholder(tf.uint8, [None, None], name='X')    # [ BATCHSIZE, SEQLEN ]
Xo = tf.one_hot(X, ALPHABETSIZE, 1.0, 0.0)                 # [ BATCHSIZE, SEQLEN, ALPHASIZE ]

# expected outputs = same sequence shifted by 1 since we are trying to predict the next character
Y_ = tf.placeholder(tf.uint8, [None, None], name='Y_')  # [ BATCHSIZE, SEQLEN ]
Yo_ = tf.one_hot(Y_, ALPHABETSIZE, 1.0, 0.0)               # [ BATCHSIZE, SEQLEN, ALPHASIZE ]

# input state
Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name='Hin')  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

# using NLAYERS layers of GRU cells, unrolled SEQLEN=30 times
# dynamic_rnn infers SEQLEN from the size of the inputs Xo

cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]

# "naive dropout" implementation
dropcells = [rnn.DropoutWrapper(cell,input_keep_prob=keep_prob_placeholder) for cell in cells]
multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = rnn.DropoutWrapper(multicell, output_keep_prob=keep_prob_placeholder)  # dropout for the softmax layer

Yr, H = tf.nn.dynamic_rnn(cell, Xo, dtype=tf.float32, initial_state=Hin)
# Yr: [ BATCHSIZE, SEQLEN, INTERNALSIZE ]
# H:  [ BATCHSIZE, INTERNALSIZE*NLAYERS ] # this is the last state in the sequence


# Softmax layer implementation:
# Flatten the first two dimension of the output [ BATCHSIZE, SEQLEN, ALPHASIZE ] => [ BATCHSIZE x SEQLEN, ALPHASIZE ]
# then apply softmax layer. This way, the weights and biases are shared across unrolled time steps.

W = tf.Variable(tf.random_normal([INTERNALSIZE, ALPHABETSIZE]))
B = tf.Variable(tf.random_normal([ALPHABETSIZE]))

Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
Ylogits = tf.matmul(Yflat, W) + B     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]

Yflat_ = tf.reshape(Yo_, [-1, ALPHABETSIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]

loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]

Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE, SEQLEN ]

train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# stats for display
seqloss = tf.reduce_mean(loss, 1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))

# for display: init the progress bar
DISPLAY_FREQ = 50
_50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
progress = txt.Progress(DISPLAY_FREQ, size=111+2, msg="Training on next "+str(DISPLAY_FREQ)+" batches")

# init
istate = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])  # initial zero input state
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
step = 0

# training loop
for x, y_, epoch in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=2):

    # train on one minibatch
    feed_dict = {X: x, Y_: y_, Hin: istate, keep_prob_placeholder: keep_prob, batchsize: BATCHSIZE}
    _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)

    # display a short text generated with the current weights and biases (every 150 batches)
    if step // 3 % _50_BATCHES == 0:
        print
        print "Generating some random text..."
        
        ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
        rh = np.zeros([1, INTERNALSIZE * NLAYERS])

        for k in range(1000):
            ryo, rh = sess.run([Yo, H], feed_dict={X: ry, keep_prob_placeholder: 1.0, Hin: rh, batchsize: 1})
            rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
            sys.stdout.write(chr(txt.convert_to_alphabet(rc)))
            ry = np.array([[rc]])

        print
        print "FINISHED GENERATING RANDOM TEXT"
        print

    # display progress bar
    progress.step(reset=step % _50_BATCHES == 0)

    # loop state around
    istate = ostate
    step += BATCHSIZE * SEQLEN
    
print
print "-----FINISHED TRAINING-----"


Loading file shakespeare/various.txt
Loading file shakespeare/loverscomplaint.txt
Loading file shakespeare/asyoulikeit.txt
Loading file shakespeare/othello.txt
Loading file shakespeare/kingjohn.txt
Loading file shakespeare/2kinghenryvi.txt
Loading file shakespeare/sonnets.txt
Loading file shakespeare/merchantofvenice.txt
Loading file shakespeare/titusandronicus.txt
Loading file shakespeare/hamlet.txt
Training text size is 0.89MB with 178.29KB set aside for validation. There will be 1554 batches per epoch

Generating some random text...
I C&`!1!`k1!&191!U!W9!w99999999v9fDCD!C1C1C99991c9":DIiIE!`k!91`kkCu`UU!9!9uv9`fUf11!99D"C"u9Uc9f9D!C9DC!`uI9C19!!9`9`k```k!`U!I99D`9``U9f`119!kCf91`!k`kCUccc`!I1Ck99`!!1!999fEk11919!v99`9!U`Uk!1!`9Di`9Y1UDY9!!C99kf99uCu99v91C99D"C"9"9C9CkY"u9U!9f1C9v99111-CC!`!9u`!99uf9!D1f!9D999v!911"9D1CCCc!999c1""u9!I9DU9c9!999!Cf9999k9u"1C""DCuUI9Uf99E9kC"9uD9UD9fU9ECCCf91CCc11!U!99`9k!`9U!`19!!`UI(f9k!`YkY !II9D Cc9`191U!91`9D`D!91kY19k!CU1CU!9k1!kIDC919!!u9!I!9D11

	Th up ave tit im shat is,
	HE was tho sere.

IA	The sto surcerish aicknore dodiston dowist west me meale prow whlt youn seat of an wothe pravend.

TEMURKIIUS	Yow ou thar to hat hisper, in the wishe s arking wor wnan ckntighas beitht yours:
	Th merdear thute thed frout sors
	Ang at mires ar trye on and oup thy htod ma
FINISHED GENERATING RANDOM TEXT

()
0%                                        Training on next 50 batches                                        100%
=================================================================================================================()
0%                                        Training on next 50 batches                                        100%
=================================================================================================================()
0%                                        Training on next 50 batches                                        100%
Generating some random text...
I9DG ES	Wait have tyou have sache pange

	Theat bo thas a bongaight offils; more forcout hear oo for
	For the hould and ane in of thy heres mist the full and
	Ino my ag on mosh then soon the fee moke miness.
	Him spee s my praves: weine bofer ig mo hid briand.
	Buepad, the kill where fill I so my whe to a woon thin ana ither andiciors
	The der of year hy bent of muce.

IAGO	Indo my have to heres is and forrown that heen suck?
Thar conese of then of onor by theie,
	Whal is im migringin ond stelough hin dom herse.
	Toor's to sark woll ond my bleases: to shourd, mo nom
	I came one, wom the wame:)

ARONIO	Leakenot I sempens afly sor my poord mouress.

DoKING JAQUEN	Yrupes, be poret of thesest ald bende
	Tion in as ar hte helled till thr eapon.
	But and bet thou men my frell wo highinst mart, It wiclly, anl benthis therest to lot ald heme;
I have I and in these fot I dos brovesthithinge
	Thee a draith on
FINISHED GENERATING RANDOM TEXT

()
0%                                        Training on next 50 batches                       

=================================================================================================================()
0%                                        Training on next 50 batches                                        100%
Generating some random text...
I	NI If aly waup,
	We mont hes laven's ale nage he speatty hind: for thath;
	Hathy of here de in you wandery a stack of yet?

Then I houp it tho grove a clayf, berdy, both wour, ho touthan.

PORTANA	[Dusionco, if in wost to bere, now, that bastay
	Of fairun, to to here sere hin;
	This ablotion a cimply:--lake,: but thee wicl.

	[liekn thearing in batter thy hear.

AQUERIIN	Sheapon'd for som in thisk that be bustayou,
	Then of all there my swill thre fairn thich ale the coon,
	Or atast not tale and with mine, and wort;
	Mer sore the wighiflis hare have her ond
	And here bust, we loveds thee wonds, he winder here,
I samy twere it ate see thee swar holothod thee frive:
	Ay, beidife home all to kingent be chater ateress: my my lead;


	Art, a goont to and blyed all tere,
	Suce, I am west what those a fothing mery more a trought?

	[Enter this curpor of is is there by swiet a mo,
	To to dose. O, f
FINISHED GENERATING RANDOM TEXT

()
0%                                        Training on next 50 batches                                        100%
=================================================================================================================()
0%                                        Training on next 50 batches                                        100%
=================================================================================================================()
0%                                        Training on next 50 batches                                        100%
Generating some random text...
ike!
	IThen hast now, and so waund,
Mons when sumald but hand hears hose of. O,
LONDEve I seen ling and thou suse;
Byan of much must old, tone athendose as to do wo see

	[Exitut of Shill our sa